In [13]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import gkey

In [2]:
#Load previous csv
weather_df = pd.read_csv('output_data/cities.csv',index_col=0)

In [3]:
weather_df.head()

,City,Temperature(F),Latitude,Longitude,Humidity,Cloudiness,Wind Speed
0,yablonovo,64.08,50.8523,37.3574,75,57,6.33
1,rikitea,74.03,-23.1203,-134.9692,80,100,17.22
2,mount gambier,54.21,-37.8333,140.7667,58,100,13.42
3,lasa,62.37,46.6166,10.7002,88,6,1.74
4,port alfred,60.19,-33.5906,26.8910,86,55,5.01


In [4]:
#Configure gmaps with API key
gmaps.configure(api_key=gkey)

In [5]:
#create heatmap variables
locations = weather_df[['Latitude','Longitude']].astype(float)
humidity = weather_df['Humidity'].astype(float)

In [6]:
#create humidity heatmap
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations,weights = humidity, dissipating = False, max_intensity = 100, point_radius = 5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
#Weather Criteria
high_temp = 75
low_temp = 65
max_wind_speed = 10
max_humidity = 50

In [8]:
#Ideal Weather DataFrame
ideal_weather_df = weather_df.loc[(weather_df['Temperature(F)'] > low_temp)\
                                  & (weather_df['Temperature(F)'] < high_temp)\
                                  & (weather_df['Wind Speed'] < max_wind_speed)\
                                  & (weather_df['Humidity'] < max_humidity),:]
ideal_weather_df.reset_index(drop=True, inplace=True)
ideal_weather_df

,City,Temperature(F),Latitude,Longitude,Humidity,Cloudiness,Wind Speed
0,iki-burul,71.65,45.8208,44.6472,40,0,3.00
1,emerald,66.34,-23.5333,148.1667,40,0,8.77
2,mount isa,74.97,-20.7333,139.5000,25,5,9.22
3,east london,65.08,-33.0153,27.9116,43,63,5.03
4,taltal,67.59,-25.4000,-70.4833,38,0,3.69
5,hovd,72.12,48.0056,91.6419,33,42,1.59
6,airai,72.81,-8.9266,125.4092,48,10,4.94
7,sao felix do xingu,74.52,-6.6447,-51.9950,46,6,1.12
8,murgab,71.47,37.4966,61.9714,43,0,6.91
9,narimanov,66.11,46.6927,47.8498,42,1,5.88


In [9]:
#Get marker locations from ideal weather dataset
marker_locations = ideal_weather_df[['Latitude','Longitude']].astype(float)

In [32]:
#Loop through ideal weather df to find nearby hotels, and store into hotel list
target_type = "hotel"
radius = 5000
target_search = "hotel"
hotel_list = []

for index, row in ideal_weather_df.iterrows():
    coordinates = (f"{ideal_weather_df['Latitude'][index]}, {ideal_weather_df['Longitude'][index]}")
    
    params = {
    "location": coordinates,
    "keyword": target_search,
    "type": target_type,
    "radius": radius,
    "key": gkey
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    hotel_response = requests.get(base_url, params).json()
    try:
        hotel_list.append(hotel_response['results'][0]['name'])
    except IndexError:
        hotel_list.append('No nearby hotels')
   
    
    
hotel_list    
    
    

['No nearby hotels',
 'Emerald Star Hotel',
 'ibis Styles Mt Isa Verona',
 'Garden Court East London',
 'Hotel Gali',
 'Steppe hotel',
 'No nearby hotels',
 'Dumont Palace Hotel',
 'No nearby hotels',
 'Solnechnyi',
 'No nearby hotels',
 'No nearby hotels']

In [33]:
#add hotel list to dataframe
ideal_weather_df['Hotels'] = hotel_list
ideal_weather_df

C:\Users\ericb\AppData\Local\Temp\ipykernel_11228\3134934419.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_weather_df['Hotels'] = hotel_list


,City,Temperature(F),Latitude,Longitude,Humidity,Cloudiness,Wind Speed,Hotels
0,iki-burul,71.65,45.8208,44.6472,40,0,3.00,No nearby hotels
1,emerald,66.34,-23.5333,148.1667,40,0,8.77,Emerald Star Hotel
2,mount isa,74.97,-20.7333,139.5000,25,5,9.22,ibis Styles Mt Isa Verona
3,east london,65.08,-33.0153,27.9116,43,63,5.03,Garden Court East London
4,taltal,67.59,-25.4000,-70.4833,38,0,3.69,Hotel Gali
5,hovd,72.12,48.0056,91.6419,33,42,1.59,Steppe hotel
6,airai,72.81,-8.9266,125.4092,48,10,4.94,No nearby hotels
7,sao felix do xingu,74.52,-6.6447,-51.9950,46,6,1.12,Dumont Palace Hotel
8,murgab,71.47,37.4966,61.9714,43,0,6.91,No nearby hotels
9,narimanov,66.11,46.6927,47.8498,42,1,5.88,Solnechnyi


In [37]:
#create humidity heatmap
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations,weights = humidity, dissipating = False, max_intensity = 100, point_radius = 5)

fig.add_layer(heat_layer)

markers = gmaps.marker_layer(marker_locations,
                            info_box_content = [f"Hotel: {hotel}" for hotel in ideal_weather_df['Hotels']])
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))